In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size=3, padding=1, activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2, strides=2))
    return blk

In [6]:
conv_arch = ((1, 32), (1, 32), (2, 64), (2, 128), (2, 256))

def vgg(conv_arch):
    net = nn.Sequential()
    for (num_convs, num_channnels) in conv_arch:
        net.add(vgg_block(num_convs, num_channnels))
    net.add(nn.Dense(1024, activation='relu'), nn.Dropout(0.5),
            nn.Dense(1024, activation='relu'), nn.Dropout(0.5),
            nn.Dense(10))
    return net

net = vgg(conv_arch)

In [7]:
net.initialize()
X = nd.random.uniform(shape=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

sequential13 output shape:	 (1, 32, 112, 112)
sequential14 output shape:	 (1, 32, 56, 56)
sequential15 output shape:	 (1, 64, 28, 28)
sequential16 output shape:	 (1, 128, 14, 14)
sequential17 output shape:	 (1, 256, 7, 7)
dense6 output shape:	 (1, 1024)
dropout4 output shape:	 (1, 1024)
dense7 output shape:	 (1, 1024)
dropout5 output shape:	 (1, 1024)
dense8 output shape:	 (1, 10)


In [8]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [9]:
lr, num_epochs, ctx = 0.05, 5, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 0.8785, train acc 0.676, test acc 0.840, time 84.3 sec
epoch 2, loss 0.4365, train acc 0.842, test acc 0.885, time 82.3 sec
epoch 3, loss 0.3617, train acc 0.868, test acc 0.891, time 82.4 sec
epoch 4, loss 0.3231, train acc 0.883, test acc 0.900, time 83.3 sec
epoch 5, loss 0.2951, train acc 0.893, test acc 0.904, time 83.1 sec
